<a href="https://colab.research.google.com/github/ngochif/fngochi_64061/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the IMDB Dataset**

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  15.6M      0  0:00:05  0:00:05 --:--:-- 17.2M


In [25]:
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb/train/unsup': No such file or directory


In [26]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

**Data Preparation**

In [80]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):

    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.5 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [81]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 10000 files belonging to 2 classes.
Found 15000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Defining the parameters for the new requirements**

In [87]:
max_features = 10000  # Consider only the top 10,000 words
sequence_length = 150  # Cutoff reviews after 150 words
training_samples = 100  # Number of training samples
validation_samples = 10000  # Number of validation samples
batch_size = 25  # Batch size for training

**Preparing the integer sequence datasets according to new requirements**



In [89]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential

# Load datasets
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    shuffle=True
)
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/val",
    batch_size=batch_size,
    shuffle=True
)
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)

Found 10000 files belonging to 2 classes.
Found 15000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [90]:
# Limit the dataset to an exact number of samples
def limit_dataset(ds, num_samples):
    return ds.unbatch().take(num_samples).batch(batch_size)

# Apply the limiting function
train_ds = limit_dataset(raw_train_ds, training_samples)
val_ds = limit_dataset(raw_val_ds, validation_samples)
test_ds = raw_test_ds
# Text Vectorization
text_vectorization = TextVectorization(
    max_tokens=max_features,
    output_sequence_length=sequence_length
)
train_text = train_ds.map(lambda x, y: x)
text_vectorization.adapt(train_text)

In [91]:
# Slice the datasets
train_ds = raw_train_ds.take(training_samples // batch_size)
val_ds = raw_val_ds.take(validation_samples // batch_size)
test_ds = raw_test_ds

# Text Vectorization
text_vectorization = TextVectorization(
    max_tokens=max_features,
    output_sequence_length=sequence_length
)
train_text = train_ds.map(lambda x, y: x)
text_vectorization.adapt(train_text)

In [92]:
def calculate_dataset_size(dataset):
    size = 0
    for batch in dataset:
        size += tf.shape(batch[0])[0]  # count the number of items in the batch
    return size

# Calculate the sizes
train_size = calculate_dataset_size(train_ds)
val_size = calculate_dataset_size(val_ds)
test_size = calculate_dataset_size(test_ds)

# Print the sizes
print(f"Training Dataset Size: {train_size}")
print(f"Validation Dataset Size: {val_size}")
print(f"Test Dataset Size: {test_size}")

Training Dataset Size: 100
Validation Dataset Size: 10000
Test Dataset Size: 25000


In [93]:
# Vectorize the datasets
train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
test_ds = raw_test_ds.map(lambda x, y: (text_vectorization(x), y))

**Training a Model that uses an Embedding layer trained from scratch**

In [94]:
from tensorflow.keras import layers
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_features, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(test_ds)[1]:.3f}")

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_5 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

**Training a Model Using GLOVE Pretrained Embedding**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

Parsing the GloVe word-embeddings file

In [67]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [68]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [69]:
embedding_layer = layers.Embedding(
    max_features,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)


Preparing the GloVe word-embeddings matrix

In [96]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(test_ds)[1]:.3f}")

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_9 (Embedding)     (None, None, 100)         1000000   
                                                                 
 bidirectional_6 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1034113 (3.94 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 1000000 (3.81 MB)
_________________